In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras
from tcn import tcn
from sklearn.model_selection import train_test_split


sc = StandardScaler()

In [47]:
dfs = []
for i in range(3,6):
    df = pd.read_pickle("../data/df_MB_"+str(i))
    df.drop("sensor", axis=1, inplace=True)
    dfs.append(df)

train = pd.concat(dfs, axis=1)
print(train.shape)
train.columns = [sensor+m for sensor in ["acc_", "gyro_", "mag_"] for m in ["x", "y", "z"]]
#train["y"] = 0

dfs = []
for i in range(3,6):
    df = pd.read_pickle("../data/df_not_squats_"+str(i))
    df.drop("sensor", axis=1, inplace=True)
    dfs.append(df)

train2 = pd.concat(dfs, axis=1)
train2.columns = [sensor+m for sensor in ["acc_", "gyro_", "mag_"] for m in ["x", "y", "z"]]
#train2["y"] = 1
print(train2.shape)
train = pd.concat([train,train2], ignore_index=True, axis=0)
print(train.shape)
train.drop("mag_z", axis=1, inplace=True)
train.reset_index(drop=True, inplace=True)
train.fillna(method="pad", inplace=True)
train

(4500, 9)
(4500, 9)
(9000, 9)


,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,mag_x,mag_y
0,1.935,-5.104,0.690,3.069,0.177,0.579,-15.125,58.500000
1,-5.784,-7.403,3.266,2.771,-0.469,0.390,-13.813,56.500000
2,-0.881,-5.650,5.104,2.677,-0.504,-0.155,-14.000,53.063000
3,0.019,-3.792,7.518,2.652,-1.042,-0.089,-12.813,48.250000
4,0.163,-2.835,8.389,2.376,-1.106,-0.100,-13.750,41.938000
5,-1.025,-2.145,9.318,2.195,-1.017,-0.303,-16.125,34.188000
6,0.766,0.124,10.218,1.957,-1.172,-0.411,-17.063,27.188000
7,1.398,0.555,10.898,1.005,-0.634,-0.366,-20.938,17.313000
8,2.538,2.950,11.837,-0.210,0.196,0.181,-22.500,11.688000
9,3.026,4.070,13.819,-1.679,0.627,0.639,-26.250,5.250000


In [48]:
train_sc = sc.fit_transform(train.values.astype("float"))
train_sc.shape

(9000, 8)

In [52]:
splits = np.array_split(train_sc, len(train)/45)

In [53]:
X = np.stack(splits)

In [78]:
X.shape

(200, 45, 8)

In [85]:
y = np.zeros(len(X))
y[:100] = 1

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [87]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160, 45, 8), (40, 45, 8), (160,), (40,))

In [88]:
model, param_str = tcn.dilated_tcn(output_slice_index='last', # try 'first'.
                                       num_feat=8,
                                       num_classes=2,
                                       nb_filters=8,
                                       kernel_size=4,
                                       dilatations=[1, 2, 4, 8],
                                       nb_stacks=8,
                                       max_len=45,
                                       activation='norm_relu',
                                       use_skip_connections=False,
                                       return_param_str=True)

#print(f'x_train.shape = {x_train.shape}')
#print(f'y_train.shape = {y_train.shape}')
# Using sparse softmax.
# http://chappers.github.io/web%20micro%20log/2017/01/26/quick-models-in-keras/
model.summary()


x.shape= (?, 8)
model.x = (?, 45, 8)
model.y = (?, 2)
Adam with norm clipping.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 45, 8)        0                                            
__________________________________________________________________________________________________
initial_conv (Conv1D)           (None, 45, 8)        264         input_layer[0][0]                
__________________________________________________________________________________________________
dilated_conv_2_tanh_s0 (Conv1D) (None, 45, 8)        264         initial_conv[0][0]               
__________________________________________________________________________________________________
activation_153 (Activation)     (None, 45, 8)        0           dilated_conv_2_tanh_s0[0][0]     
______________________________

In [89]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=10,batch_size=4, )

Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 71s 446ms/step - loss: 0.6146 - acc: 0.7625 - val_loss: 0.1312 - val_acc: 0.9500
Epoch 2/10
160/160 [==============================] - 68s 428ms/step - loss: 0.2829 - acc: 0.9250 - val_loss: 0.0639 - val_acc: 0.9750
Epoch 3/10
160/160 [==============================] - 67s 421ms/step - loss: 0.1943 - acc: 0.9375 - val_loss: 0.2734 - val_acc: 0.9500
Epoch 4/10
160/160 [==============================] - 75s 467ms/step - loss: 0.2502 - acc: 0.9688 - val_loss: 0.3039 - val_acc: 0.9500
Epoch 5/10
160/160 [==============================] - 77s 479ms/step - loss: 0.2078 - acc: 0.9562 - val_loss: 0.2985 - val_acc: 0.9500
Epoch 6/10
160/160 [==============================] - 72s 448ms/step - loss: 0.1485 - acc: 0.9500 - val_loss: 0.2396 - val_acc: 0.9500
Epoch 7/10
160/160 [==============================] - 75s 470ms/step - loss: 0.0420 - acc: 0.9813 - val_loss: 0.1794 - val_acc: 0.9500
Epoch 8/10